In [1]:
import pandas as pd
from dateutil import parser
from arche.helpers import path2cols, CollectionBuilder

In [2]:
file = "documents/data/thunau_sample_data.xlsx"
prefix = "https://id.acdh.oeaw.ac.at/thunau/top-collection"

In [3]:
df = pd.read_excel(file)

In [4]:
troubles = []
for index, row in df.iterrows():
    doc, _ = Document.objects.get_or_create(legacy_id=row['ID'])
    doc.filename = row['Dateiname']
    doc.entry_order = row['Ordnungskriterium/Eingabe']
    vocabs_media, _ = SkosConcept.objects.get_or_create(pref_label=row['Medium'])
    vocabs_media_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='Medium')
    vocabs_media.scheme.set([vocabs_media_scheme])
    vocabs_media.save()
    doc.medium = vocabs_media
    vocabs_analogformat, _ = SkosConcept.objects.get_or_create(pref_label=row['Analoges Format'])
    vocabs_analogformat_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='Analoges Format')
    vocabs_analogformat.scheme.set([vocabs_analogformat_scheme])
    vocabs_analogformat.save()
    doc.analogue_format = vocabs_analogformat
    try:
        names = row['written Autor'].split(';')
    except:
        names = row['written Autor']
    try:
        for x in names:
            author, _ = Person.objects.get_or_create(written_name=x)
            doc.author.add(author)
            doc.save()
    except:
        troubles.append({'id': row['ID'], 'troublefield': 'Autor', 'value': row['written Autor']})
    
    institution, _ = Institution.objects.get_or_create(written_name=row['Institution'])
    doc.institution.add(institution)
    doc.date_analogue = row['written Datum']
    try:
        doc.date_digitization = row['Datum der Digitalisierung']
    except:
        troubles.append({'id': row['ID'], 'troublefield': row['Datum der Digitalisierung']})
    vocabs_digital_format, _ = SkosConcept.objects.get_or_create(pref_label=row['Speicherformat'])
    vocabs_digital_format_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='Speicherformat')
    vocabs_digital_format.scheme.set([vocabs_digital_format_scheme])
    vocabs_digital_format.save()
    doc.digital_format = vocabs_digital_format
    doc.note = row['Anmerkung']
    doc.content = row['Inhalt']
    vocabs_group, _ = SkosConcept.objects.get_or_create(pref_label=row['Gruppe'])
    vocabs_group_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='Gruppe')
    vocabs_group.scheme.set([vocabs_group_scheme])
    vocabs_group.save()
    doc.topic_group = vocabs_group
    doc.combination = row['Kombination']
    doc.location_id = row['Fundnummer in FDB']
    temp_place, _ = Place.objects.get_or_create(name=row['KG/Areal'])
    doc.place_of_origin = temp_place
    doc.location_digitized_object = row['Aufbewahrung Datei']
    doc.location_analogue = row['Standort analog']
    curat_name = row['Bearbeiter Digitalisierung']
    temp_curator, _ = Person.objects.get_or_create(written_name=curat_name)
    temp_curator.save()
    doc.curator = temp_curator
    doc.filesize = row['Dateigröße KB']
    temp_dig_inst, _ = Institution.objects.get_or_create(written_name=row['Ort der Digitalisierung'])
    doc.place_digizization = temp_dig_inst
    doc.path = row['OREA_Doku_Plattform Thunau am Kamp_Dateipfad']
    col_builder = CollectionBuilder(row['OREA_Doku_Plattform Thunau am Kamp_Dateipfad'], prefix)
    cols = col_builder.make_collections()
    doc.in_collection = Collection.objects.get(
        has_id=cols[-1]['has_id']
    )
    doc.amendments = row['Ergänzungen']
    try:
        doc.save()
    except ValueError:
        print(doc)
    doc.path = row['OREA_Doku_Plattform Thunau am Kamp_Dateipfad']

Schnittübersicht 1989


In [7]:
# add infos to already created persons

In [8]:
file = "documents/data/OREA Liste Personen.xlsx"

In [9]:
df = pd.read_excel(file)

In [10]:
for index, row in df.iterrows():
    try:
        per = Person.objects.get(written_name=row["written Autor"])
        gnd = row['nach Normdatei Deutsche Nationalbibliothek']
        if isinstance(gnd, str):
            per.authority_url = gnd
        per.name = row['Nachname']
        per.forename = row['Vorname']
        per.acad_title= row['Titel/Anrede']
        per.comment= row['Kommentare']
        inst = row['zugehörige Institution']
        try:
            if isinstance(inst, str):
                temp_inst = Institution.objects.get(written_name=inst)
                per.belongs_to_institution = temp_inst
        except:
            pass
        per.save()
    except:
        pass

In [ ]:
Institution.objects.all().delete()

In [ ]:
import json

In [ ]:
with open('troubles.json', 'w') as fp:
    json.dump(troubles, fp)

In [12]:
Document.objects.all().delete()

(142,
 {'documents.Document': 37,
  'documents.Document_author': 30,
  'documents.Document_institution': 38,
  'idprovider.IdProvider': 37})

In [ ]:
# Collection.objects.all().delete()